# Demonstration of hscScore model
This notebook demonstrates how to load data for input into the hscScore model

The model, gene list and test data can all be downloaded from [Zenodo](https://doi.org/10.5281/zenodo.3332151) (DOI: 10.5281/zenodo.3332151)

In [1]:
import pandas as pd
import numpy as np
import pickle
import sklearn
import platform

data_dir = './Zenodo_data/'

Define function for total count normalisation

In [2]:
def total_count_normalise(count_matrix):
    """Normalise count matrix for input into hscScore model.
    Performs read depth normalisation normalising each cell so that normalised 
    counts sum to the same value.
    
    Parameters
    ----------
    count_matrix : pandas dataframe
        Gene count matrix of dimension cells x genes with column names as genes
        and index as cell names
    
    Returns
    -------
    **norm_matrix** : pandas dataframe
        Normalised count matrix of dimension cells x genes
    """
    
    # Set the value normalised counts will sum to for each cell
    wilson_molo_genes_median_counts = 18704.5
    
    # Scale rows
    count_matrix_expression = np.array(count_matrix, dtype='float')
    counts_per_cell = np.sum(count_matrix_expression, axis=1)
    counts_per_cell += (counts_per_cell == 0)
    counts_per_cell /= wilson_molo_genes_median_counts
    norm_matrix_expression =  count_matrix_expression/counts_per_cell[:, None]
    norm_matrix = pd.DataFrame(norm_matrix_expression, index=count_matrix.index,
                               columns=count_matrix.columns)
    # log + 1 transform the data
    norm_matrix = np.log(norm_matrix + 1)
    
    return norm_matrix

Load the trained model. This can be downloaded from Zenodo.

In [3]:
hsc_score = pickle.load(open(data_dir + 'hscScore_model.pkl', 'rb'))

Load some test data. This should be a count matrix of cells x genes. Columns must be labelled with common gene names. Here we are using data from [Nestorowa et al. (2016)]( https://doi.org/10.1182/blood-2016-05-716480) that can be downloaded from [Zenodo](https://doi.org/10.5281/zenodo.3332151) to reproduce this example.

In [4]:
count_data = pd.read_csv(data_dir + 'nestorowa_htseq_counts.txt', sep=' ').T
count_data.head()

,Gnai3,Pbsn,Cdc45,H19,Scml2,Apoh,Narf,Cav2,Klf6,Scmh1,...,RP24-571H12.6,RP24-246N21.1,RP23-301H20.2,RP24-103L16.5,RP23-149H20.3,RP24-343A12.4,RP23-408A1.4,RP23-8L20.8,RP24-351O18.4,RP23-57N5.3
HSPC_001,232,0,2,0,0,0,0,0,56,0,...,0,3179,0,0,0,0,0,16,0,0
HSPC_002,0,0,4,0,0,0,1,0,0,0,...,0,117,0,0,0,0,0,0,0,0
HSPC_003,478,0,693,1,0,0,1,0,2,201,...,0,1593,0,0,0,0,0,7,2,0
HSPC_004,6,0,4,0,1,0,205,0,1,1,...,0,259,0,0,0,0,1,5,0,0
HSPC_005,2,0,0,0,0,0,2,0,2,0,...,0,10,0,0,0,0,0,0,2,0


Subset to the same genes in the same order as used for training the model. This list can be downloaded from Zenodo.

In [5]:
model_genes = np.genfromtxt(data_dir + 'model_molo_genes.txt', dtype='str')
model_genes

array(['2810417H13Rik', 'Aqp1', 'Aqp9', 'Arhgap27', 'Arhgap5', 'Asf1b',
       'Aspm', 'Birc5', 'Cacybp', 'Ccna2', 'Ccne2', 'Cd82', 'Cdca8',
       'Cdk6', 'Cdkn1c', 'Cenpf', 'Cenph', 'Cenpi', 'Ckap2', 'Ckap2l',
       'Cldn10', 'Csgalnact1', 'Ctsf', 'Dlgap5', 'Dtnbp1', 'Fads3',
       'Fam64a', 'Fgfr3', 'Gata1', 'Gimap1', 'Gimap6', 'Gja1', 'Glipr1',
       'Gp1bb', 'Gp5', 'Gpd2', 'Gstm1', 'Hmmr', 'Hsp90aa1', 'Ifitm1',
       'Ints2', 'Itga2b', 'Kif2c', 'Kif4', 'Knstrn', 'Limd2', 'Ltb',
       'Ly6a', 'Mdm1', 'Mettl7a1', 'Mfsd2b', 'Mis18bp1', 'Mki67', 'Mllt3',
       'Muc13', 'Nasp', 'Ncapg', 'Ndrg1', 'Neil2', 'Nek2', 'Neo1', 'Nkg7',
       'Nuf2', 'Pa2g4', 'Pbk', 'Pde1b', 'Pdzk1ip1', 'Pf4', 'Plk1',
       'Procr', 'Ptpn14', 'Ramp2', 'Rasa2', 'Rasal3', 'Rnf168', 'Rrm1',
       'Sdf2l1', 'Serpinb1a', 'Sgol1', 'Sgpp1', 'Sh2d3c', 'Sh2d5',
       'Shcbp1', 'Ska1', 'Slc22a3', 'Smtnl1', 'Sox18', 'Spag5', 'Spc24',
       'Spred1', 'Sqrdl', 'Sult1a1', 'Syk', 'Top2a', 'Tor1b', 'Trim47',
       

In [6]:
count_data_molo = count_data[model_genes]
count_data_molo.head()

,2810417H13Rik,Aqp1,Aqp9,Arhgap27,Arhgap5,Asf1b,Aspm,Birc5,Cacybp,Ccna2,...,Top2a,Tor1b,Trim47,Trip13,Ube2c,Ubl3,Uhrf1,Vps18,Vwf,Yme1l1
HSPC_001,1,2,0,0,0,3,4,5,1064,7,...,22,588,0,2,12,1227,2,535,2,1709
HSPC_002,5,6,1,0,1,1,1,4,3,6,...,8,0,150,1,6,5,1,0,61,191
HSPC_003,207,3,0,39,0,69,5,8,118,2,...,217,4,0,1,4,250,658,0,1,423
HSPC_004,7,4,0,0,1,0,7,3,202,3,...,9,2,0,0,4,2,2,1,7,14
HSPC_005,1,5,0,1,0,0,3,5,5,4,...,5,1,0,1,5,0,1,1,1,5


Normalise for model input

In [7]:
normalised_data_molo = total_count_normalise(count_data_molo)
normalised_data_molo.head()

,2810417H13Rik,Aqp1,Aqp9,Arhgap27,Arhgap5,Asf1b,Aspm,Birc5,Cacybp,Ccna2,...,Top2a,Tor1b,Trim47,Trip13,Ube2c,Ubl3,Uhrf1,Vps18,Vwf,Yme1l1
HSPC_001,0.370214,0.639793,0.000000,0.000000,0.000000,0.851915,1.026821,1.175641,6.169023,1.419804,...,2.384807,5.577652,0.000000,0.639793,1.852624,6.311281,0.639793,5.483566,0.639793,6.642105
HSPC_002,2.927585,3.100945,1.512154,0.000000,1.512154,1.512154,1.512154,2.717734,2.451821,3.100945,...,3.377312,0.000000,6.275654,1.512154,3.100945,2.927585,1.512154,0.000000,5.378634,6.516888
HSPC_003,5.219674,1.298784,0.000000,3.573550,0.000000,4.131821,1.694036,2.092634,4.661711,1.021212,...,5.266603,1.515812,0.000000,0.635666,1.515812,5.407485,6.372445,0.000000,0.635666,5.931561
HSPC_004,3.247919,2.717028,0.000000,0.000000,1.511564,0.000000,3.247919,2.451130,6.572047,2.451130,...,3.490562,2.087861,0.000000,0.000000,2.717028,2.087861,2.087861,1.511564,3.247919,3.921448
HSPC_005,4.584095,6.185329,0.000000,4.584095,0.000000,0.000000,5.675875,6.185329,6.185329,5.962700,...,6.185329,4.584095,0.000000,4.584095,6.185329,0.000000,4.584095,4.584095,4.584095,6.185329


Input into hscScore

In [8]:
predicted_hsc_scores = hsc_score.predict(np.array(normalised_data_molo))
predicted_hsc_scores

array([ 3.31854800e-01,  1.77624683e-01,  1.94555555e-02, ...,
       -5.81703918e-03,  1.09667487e-04, -3.67555679e-03])

These predicted scores can now be saved, plotted on dimensionality reductions etc...

In [9]:
print("Python version")
print(platform.python_version())

print("Scikit-learn version")
print(sklearn.__version__)

Python version
3.7.3
Scikit-learn version
0.21.2
